In [2]:
import sys
from pathlib import Path
SRC_ROOT = "../src"
if str(SRC_ROOT) not in sys.path:
    sys.path.insert(0, str(SRC_ROOT))

In [3]:
from twc.twc_io import mcc_obs_encoder, twc_out_2_mcc_action
from twc.twc_builder import build_twc, TWC
from td3 import TD3Config
import json
import gymnasium as gym
import torch
import numpy as np

/home/abanchio/micromamba/envs/PyURI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-26 12:42:33.667569: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
env = gym.make("MountainCarContinuous-v0")
env.observation_space

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)

In [14]:
config_path = Path("full_config.json")
print(config_path)
cfg = TD3Config()
if config_path.exists:
    with open(config_path, 'r') as f:
        config_data =  json.load(f)
    cfg = cfg.load(config_data)

print(cfg.to_json())

full_config.json
{
    "max_train_steps": 300000,
    "max_episode_steps": 999,
    "warmup_steps": 10000,
    "batch_size": 256,
    "num_update_loops": 2,
    "update_every": 1,
    "device": "cuda",
    "seed": 42,
    "eval_interval_episodes": 10,
    "eval_episodes": 10,
    "use_bptt": true,
    "sequence_length": 8,
    "burn_in_length": 4,
    "actor_lr": 0.0002239407231090426,
    "critic_lr": 0.0001828306017572226,
    "gamma": 0.9823522271023871,
    "tau": 0.007693135327059323,
    "policy_delay": 2,
    "target_noise": 0.28415959581368067,
    "noise_clip": 0.31789035300173857,
    "sigma_start": 0.39609107327435644,
    "sigma_end": 0.08244881107627974,
    "sigma_decay_episodes": 220,
    "twc_internal_steps": 1,
    "twc_trhesholds": [
        -0.5,
        0.0,
        0.0
    ],
    "twc_decays": [
        0.1,
        0.1,
        0.1
    ],
    "rnd_init": true,
    "use_v2": true,
    "steepness_fire": 14.434533089746672,
    "steepness_gj": 7.133187732282942,
    

In [20]:
v2_params = {}
if cfg.use_v2:
    v2_params = {
        'steepness_fire': cfg.steepness_fire,
        'steepness_gj': cfg.steepness_gj,
        'steepness_input': cfg.steepness_input,
        'input_thresh': cfg.input_thresh,
        'leaky_slope': cfg.leaky_slope,
        }

actor = build_twc(
    obs_encoder=mcc_obs_encoder,
    action_decoder=twc_out_2_mcc_action,
    internal_steps=cfg.twc_internal_steps,
    initial_thresholds=cfg.twc_trhesholds,
    initial_decays=cfg.twc_decays,
    rnd_init=cfg.rnd_init,
    use_V2=cfg.use_v2,
    log_stats=False,
    **v2_params
)
print(actor.in_layer.steepness_gj)

7.133187732282942
